<a href="https://colab.research.google.com/github/vedu-13/Codtech-Cyber-Security-/blob/main/Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import HTML

html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SecureFile Encryptor/Decryptor</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Inter', sans-serif;
        }
        .custom-file-input::-webkit-file-upload-button {
            visibility: hidden;
        }
        .custom-file-input::before {
            content: 'Select File';
            display: inline-block;
            background: #4A5568;
            color: white;
            border-radius: 0.375rem;
            padding: 0.5rem 1rem;
            outline: none;
            white-space: nowrap;
            -webkit-user-select: none;
            cursor: pointer;
            font-weight: 500;
            margin-right: 1rem;
        }
        .custom-file-input:hover::before {
            background: #2D3748;
        }
        .custom-file-input:active::before {
            background: #1A202C;
        }
    </style>
</head>
<body class="bg-gray-900 text-white flex items-center justify-center min-h-screen">
    <div class="w-full max-w-md mx-auto p-8 rounded-2xl bg-gray-800 shadow-2xl">
        <div class="text-center mb-8">
            <h1 class="text-3xl font-bold text-cyan-400">SecureFile</h1>
            <p class="text-gray-400">Encrypt & Decrypt your files with AES-256</p>
        </div>

        <!-- File Input -->
        <div class="mb-6">
            <label for="file-input" class="block mb-2 text-sm font-medium text-gray-300">File</label>
            <div class="flex items-center">
                 <input type="file" id="file-input" class="custom-file-input text-gray-400 text-sm w-full">
                 <span id="file-name" class="ml-4 text-sm text-gray-500 truncate">No file selected</span>
            </div>
        </div>

        <!-- Password Input -->
        <div class="mb-6">
            <label for="password" class="block mb-2 text-sm font-medium text-gray-300">Password</label>
            <input type="password" id="password" class="bg-gray-700 border border-gray-600 text-white text-sm rounded-lg focus:ring-cyan-500 focus:border-cyan-500 block w-full p-2.5" placeholder="Enter a strong password" required>
        </div>

        <!-- Action Buttons -->
        <div class="grid grid-cols-1 md:grid-cols-2 gap-4 mb-6">
            <button id="encrypt-btn" class="w-full bg-cyan-600 hover:bg-cyan-700 text-white font-bold py-2 px-4 rounded-lg transition duration-300 ease-in-out transform hover:scale-105">
                Encrypt
            </button>
            <button id="decrypt-btn" class="w-full bg-pink-600 hover:bg-pink-700 text-white font-bold py-2 px-4 rounded-lg transition duration-300 ease-in-out transform hover:scale-105">
                Decrypt
            </button>
        </div>

        <!-- Status & Download Area -->
        <div id="status-container" class="mt-6 text-center p-4 rounded-lg bg-gray-700 hidden">
            <p id="status-message" class="text-sm font-medium"></p>
            <a id="download-link" class="hidden mt-2 text-cyan-400 hover:text-cyan-300 font-bold" href="#">Download Processed File</a>
        </div>
    </div>

    <script>
        // --- DOM Element References ---
        const fileInput = document.getElementById('file-input');
        const fileNameDisplay = document.getElementById('file-name');
        const passwordInput = document.getElementById('password');
        const encryptBtn = document.getElementById('encrypt-btn');
        const decryptBtn = document.getElementById('decrypt-btn');
        const statusContainer = document.getElementById('status-container');
        const statusMessage = document.getElementById('status-message');
        const downloadLink = document.getElementById('download-link');

        let selectedFile = null;

        // --- Event Listeners ---
        fileInput.addEventListener('change', handleFileSelect);
        encryptBtn.addEventListener('click', () => processFile(true));
        decryptBtn.addEventListener('click', () => processFile(false));

        /**
         * Handles the file selection event.
         * Updates the UI to show the name of the selected file.
         */
        function handleFileSelect(event) {
            selectedFile = event.target.files[0];
            if (selectedFile) {
                fileNameDisplay.textContent = selectedFile.name;
                fileNameDisplay.classList.remove('text-gray-500');
                fileNameDisplay.classList.add('text-gray-300');
            } else {
                fileNameDisplay.textContent = 'No file selected';
                fileNameDisplay.classList.add('text-gray-500');
                fileNameDisplay.classList.remove('text-gray-300');
            }
        }

        /**
         * Main function to process the file for encryption or decryption.
         * @param {boolean} isEncrypt - True for encryption, false for decryption.
         */
        async function processFile(isEncrypt) {
            // Validate inputs
            if (!selectedFile || !passwordInput.value) {
                showStatus('Please select a file and enter a password.', 'error');
                return;
            }

            showStatus(isEncrypt ? 'Encrypting...' : 'Decrypting...', 'processing');

            try {
                // Read file content as an ArrayBuffer
                const fileBuffer = await selectedFile.arrayBuffer();

                // Derive a key from the password
                const key = await deriveKeyFromPassword(passwordInput.value);

                let processedBuffer;
                let outputFileName;

                if (isEncrypt) {
                    // --- Encryption ---
                    const iv = crypto.getRandomValues(new Uint8Array(12)); // Generate a random 96-bit IV
                    const encryptedContent = await crypto.subtle.encrypt(
                        { name: 'AES-GCM', iv: iv },
                        key,
                        fileBuffer
                    );

                    // Prepend the IV to the encrypted data. This is crucial for decryption.
                    const encryptedDataWithIv = new Uint8Array(iv.length + encryptedContent.byteLength);
                    encryptedDataWithIv.set(iv);
                    encryptedDataWithIv.set(new Uint8Array(encryptedContent), iv.length);

                    processedBuffer = encryptedDataWithIv.buffer;
                    outputFileName = `${selectedFile.name}.encrypted`;
                    showStatus('Encryption successful!', 'success');
                } else {
                    // --- Decryption ---
                    if (fileBuffer.byteLength < 12) {
                        throw new Error("Invalid encrypted file. File is too short.");
                    }

                    // Extract the IV from the beginning of the file
                    const iv = new Uint8Array(fileBuffer.slice(0, 12));
                    const encryptedContent = fileBuffer.slice(12);

                    const decryptedContent = await crypto.subtle.decrypt(
                        { name: 'AES-GCM', iv: iv },
                        key,
                        encryptedContent
                    );

                    processedBuffer = decryptedContent;
                    // Attempt to remove the .encrypted extension if it exists
                    outputFileName = selectedFile.name.endsWith('.encrypted')
                        ? selectedFile.name.slice(0, -10)
                        : `decrypted_${selectedFile.name}`;
                    showStatus('Decryption successful!', 'success');
                }

                // Create a downloadable link for the processed file
                createDownloadLink(processedBuffer, outputFileName);

            } catch (error) {
                console.error('An error occurred:', error);
                // Provide a user-friendly error message, especially for decryption failures
                const errorMessage = isEncrypt
                    ? 'Encryption failed.'
                    : 'Decryption failed. Please check your password or file integrity.';
                showStatus(errorMessage, 'error');
            }
        }

        /**
         * Derives a 256-bit AES-GCM key from a user-provided password using PBKDF2.
         * @param {string} password - The password to derive the key from.
         * @returns {Promise<CryptoKey>} A promise that resolves to the derived CryptoKey.
         */
        async function deriveKeyFromPassword(password) {
            const encoder = new TextEncoder();
            const passwordBuffer = encoder.encode(password);

            // A salt should be unique for each password, but for simplicity in this tool,
            // we use a fixed salt. For higher security applications, a unique, random salt
            // should be generated and stored with the encrypted data.
            // Here we use a static salt for simplicity of this self-contained tool.
            const salt = encoder.encode('a-very-secret-salt');

            // PBKDF2 is used to make the key derivation process slow, protecting against brute-force attacks.
            const importedKey = await crypto.subtle.importKey(
                'raw',
                passwordBuffer,
                { name: 'PBKDF2' },
                false,
                ['deriveKey']
            );

            return crypto.subtle.deriveKey(
                {
                    name: 'PBKDF2',
                    salt: salt,
                    iterations: 250000, // A high number of iterations increases security
                    hash: 'SHA-256'
                },
                importedKey,
                { name: 'AES-GCM', length: 256 }, // Specify AES-GCM with a 256-bit key
                true,
                ['encrypt', 'decrypt']
            );
        }

        /**
         * Creates a Blob from the processed data and generates a downloadable link.
         * @param {ArrayBuffer} buffer - The data buffer of the processed file.
         * @param {string} fileName - The name for the downloaded file.
         */
        function createDownloadLink(buffer, fileName) {
            const blob = new Blob([buffer]);
            const url = URL.createObjectURL(blob);
            downloadLink.href = url;
            downloadLink.download = fileName;
            downloadLink.classList.remove('hidden');
        }

        /**
         * Displays a status message to the user.
         * @param {string} message - The message to display.
         * @param {'processing'|'success'|'error'} type - The type of message, for styling.
         */
        function showStatus(message, type) {
            statusContainer.classList.remove('hidden');
            statusMessage.textContent = message;

            // Reset styles
            statusMessage.classList.remove('text-yellow-300', 'text-green-400', 'text-red-400');
            downloadLink.classList.add('hidden'); // Hide download link initially

            // Apply new styles based on type
            if (type === 'processing') {
                statusMessage.classList.add('text-yellow-300');
            } else if (type === 'success') {
                statusMessage.classList.add('text-green-400');
            } else if (type === 'error') {
                statusMessage.classList.add('text-red-400');
            }
        }
    </script>
</body>
</html>
"""

display(HTML(html_content))